In [2]:
!conda install -y attrdict

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/Braket

  added / updated specs:
    - attrdict


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    attrdict-2.0.1             |             py_0          10 KB  conda-forge
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py37h89c1867_0         150 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         305 KB

The following NEW packages will be INSTALLED:

  attrdict           conda-forge/noarch::attrdict-2.0.1-py_0

The following packages will be UPDATED:

  ca-certificates    

## 定式化

### インデックス

- $i, j \in \mathcal{N}$: 位置を表すインデックス
- $a \in \mathcal{N}$: 紙片を表すインデックス

### 定数

- $C_{a, b, i,j}$ 位置 $i$ と $j$ が方向 $d$ で隣り合っている時 $1$、それ以外 $0$ となる定数
    - $a$ が $i$、 $b$ が $j$ に置くのにかかるコスト
    
### 変数

- $x_{a,i}$: 紙片 $a$ が位置 $i$ に置かれる時 1、置かれない時 0

### 目的関数

- $\min_x\ \sum_{a,b,i,j}C_{a, b, i,j}x_{a,i}x_{b,j}$

### 制約

- $\sum_{i}x_{a,i} = 1$: 紙片 $a$ は必ずどこか一箇所に置かれる
- $\sum_{a}x_{a,i} = 1$: 位置 $i$ に置かれる紙片は1枚

## 制約の除去

目的関数を

$\min_x\ \sum_{a,b,i,j}C_{a, b, i,j}x_{a,i}x_{b,j} + M\sum_{a}(\sum_{i}x_{a,i} - 1)^2 + M\sum_{i}(\sum_{a}x_{a,i} - 1)^2$

とすればよい。 $M$ は

$M \geq \max D_{d,a,b}$

を満たす適当に大きな定数。

<!--
もしくは、位置$i$ を行と列 $(r, c)$ と書いて、新たな補助変数 $z_{a,r}$ を導入して、


$\min_x\ \sum_{a,b,i,j}D_{d,a,b}C_{d, i,j}x_{a,i}x_{b,j} + M_1\sum_{a,r}(\sum_{c}x_{a,r,c} - z_{a,r})^2 + M_2\sum_{a}(\sum_{r}z_{a, r} - 1)^2$

とする
-->

In [12]:
import attrdict

import numpy as np
from functools import lru_cache
from collections import defaultdict
import pathlib
import pickle


def sim(a, b, d):
    if d == 'r':
        return (a[-1, :, :][:b.shape[0]] == b[0, :, :][:a.shape[0]]).sum()
    elif d == 'c':
        return (a[:, -1, :][:, :b.shape[1]] == b[:, 0, :][:, :a.shape[1]]).sum()
    else:
        raise


def idx2node(a, i, size):
    return size*i + a


def node2idx(n, size):
    i = n//size
    a = n%size
    return a, i


def get_coeff(imgs, rows, cols):
    """
    \sum_{d}S_{abd}D_{ijd} を返す
    """
    size = rows*cols
    coeff = defaultdict(float)
    for a in range(len(imgs)):
        for b in range(len(imgs)):
            for i in range(rows*cols):
                if (i + 1)%cols > 0:
                    n = idx2node(a, i, size)
                    m = idx2node(b, i + 1, size)
                    coeff[n, m] = sim(imgs[a], imgs[b], 'c')
                if i + cols < n:
                    n = idx2node(a, i, size)
                    m = idx2node(b, i + cols, size)
                    coeff[n, m] = sim(imgs[a], imgs[b], 'r')
    return coeff

In [8]:
IMG_DIR = '../data/shred'

imgs = sorted(pathlib.Path(IMG_DIR).glob('*.pkl'))

In [9]:
with open(imgs[5], 'rb') as i_:
    data = pickle.load(i_)

print(data.rows, data.cols)

2 10


In [10]:
coeff = get_coeff(data.images, data.rows, data.cols)

## D-Wave に投入

https://qard.is.tohoku.ac.jp/T-Wave/?p=1136#PyQUBO%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E5%A0%B4%E5%90%88

In [13]:
import boto3
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dwave.system.composites import EmbeddingComposite

# get the account ID
aws_account_id = boto3.client("sts").get_caller_identity()["Account"]
# the name of the bucket
my_bucket = "amazon-braket-ohtaman"
# the name of the folder in the bucket
my_prefix = "d-wave/output"
s3_location = (my_bucket, my_prefix)

In [17]:
#device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system1")
print('Device:', device)

Device: Device('name': Advantage_system1.1, 'arn': arn:aws:braket:::device/qpu/d-wave/Advantage_system1)


In [18]:
sampler = BraketDWaveSampler(s3_location, device.arn)
sampler = EmbeddingComposite(sampler)

In [19]:
sampler.sample_qubo(coeff)

ValueError: no embedding found